In [1]:
import pandas as pd
import numpy as np

In [2]:
teams = pd.read_csv('data/mens/MTeams.csv')

In [3]:
teams_spellings = pd.read_csv('data/mens/MTeamSpellings.csv', encoding='latin-1')

In [13]:
def teamname_to_teamid(teamname):
        
    mappings = {'Milwaukee': 'WI Milwaukee',
                'WI Green Bay': 'Green Bay',
                'Boston University': 'Boston Univ',
               'Southern': 'Southern Univ',
               'Cal St. Fullerton': 'CS Fullerton',
               'Cal St. Northridge': 'CS Northridge',
               'Cal St. Bakersfield': 'CS Bakersfield',
               'Maryland Eastern Shore': 'MD E Shore',
               'Mississippi Valley St.': 'MS Valley St',
               'Illinois Chicago': 'IL Chicago',
               'Arkansas Pine Bluff': 'Ark Pine Bluff',
                'Arkansas Little Rock': 'Ark Little Rock',
               'Southeast Missouri St.': 'SE Missouri St',
               'Louisiana Monroe': 'ULM',
               'Bethune Cookman': 'Bethune-Cookman',
               'Texas A&M Corpus Chris': 'TAM C. Christi',
                'Tennessee Martin': 'TN Martin',
                'UT Rio Grande Valley': 'UTRGV'
}
    try:
        teamname = mappings[teamname]
    except KeyError:
        pass
        
    try:
        teamid = teams[teams['TeamName'] == teamname].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass

    try:
        teamid = teams[teams['TeamName'] == teamname.replace('.','')].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass

    try:
        teamid = teams[teams['TeamName'] == teamname.replace("'","")].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass
    
    if teamname.endswith('St.'):
        try:
            teamid = teams[teams['TeamName'] == teamname.rstrip('.')].iloc[0]['TeamID']
            return teamid
        except (KeyError, IndexError):
            pass

    try:
        teamid = teams_spellings[teams_spellings['TeamNameSpelling'] == teamname.lower()].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        pass
    
    try:
        teamid = teams_spellings[teams_spellings['TeamNameSpelling'] == teamname.lower().replace('.','')].iloc[0]['TeamID']
        return teamid
    except (KeyError, IndexError):
        return np.nan

In [14]:
def get_and_match_kenpoms(year=2020):
    if year == 2020:
        url = 'https://kenpom.com/index.php'
    else:
        url = f'https://kenpom.com/index.php?y={year}'
    df = pd.read_html(url)[0]
    real_cols = [df.columns[i][-1].replace('.1','') for i in range(len(df.columns))]
    df.columns = real_cols
    df = df.loc[:,~df.columns.duplicated()]
    df = df[~pd.isna(df['Team'])]
    df['Team'] = df['Team'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    df['Team'] = df['Team'].apply(lambda x: x.rstrip(' '))
    df['TeamID'] = df['Team'].apply(lambda x: teamname_to_teamid(x))
    df = df[~pd.isna(df['TeamID'])]
    return df

In [15]:
all_dfs = {}
for year in range(2002, 2021):
    df = get_and_match_kenpoms(year=year)
    all_dfs[year] = df
    print(year, df.shape)
    df.to_csv(f'data/mens/kenpoms_{year}.csv')

2002 (322, 12)
2003 (322, 12)
2004 (322, 12)
2005 (326, 12)
2006 (331, 12)
2007 (332, 12)
2008 (337, 12)
2009 (340, 12)
2010 (343, 12)
2011 (342, 12)
2012 (342, 12)
2013 (344, 12)
2014 (348, 12)
2015 (348, 12)
2016 (349, 12)
2017 (350, 12)
2018 (350, 12)
2019 (353, 12)
2020 (352, 12)
